In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!pip install datasets
!pip install py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("samsum")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
sample_dataset = dataset['train']['dialogue'][:10]
print(sample_dataset)

["Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)", 'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great', "Tim: Hi, what's up?\r\nKim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating\r\nTim: What did you plan on doing?\r\nKim: Oh you know, uni stuff and unfucking my room\r\nKim: Maybe tomorrow I'll move my ass and do everything\r\nKim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies\r\nTim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores\r\nTim: It really helps\r\nKim: thanks, maybe I'll do that\r\nTim: I also like using post-its in kaban style", "Edward: Rachel, I think I'm in ove with Bella..\r\nrachel: Dont say anything else..\r\nEdward: What do you mean??\r\nrachel: Open your fu**ing door.. I'm outside", "Sam: hey  overheard rick say something\r\nSam: i don't know 

In [ ]:
sample_summaries = dataset['train']['summary'][:10]
print(sample_summaries)

['Amanda baked cookies and will bring Jerry some tomorrow.', 'Olivia and Olivier are voting for liberals in this election. ', 'Kim may try the pomodoro technique recommended by Tim to get more stuff done.', 'Edward thinks he is in love with Bella. Rachel wants Edward to open his door. Rachel is outside. ', 'Sam is confused, because he overheard Rick complaining about him as a roommate. Naomi thinks Sam should talk to Rick. Sam is not sure what to do.', "Wyatt reminds Neville his wedding anniversary is on the 17th of September. Neville's wife is upset and it might be because Neville forgot about their anniversary.", "John didn't show up for class due to some work issues with his boss. Cassandra, his teacher told him which exercises to do, and which chapter to study. They are going to meet up for a beer sometime this week after class. ", 'Sarah sends James an instrumental song he might like. James knows the song. The brain connects the songs to the context they were played in and brings 

In [ ]:
sample = sample_dataset[0]
print(sample)

Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)


In [ ]:
sample_summary = sample_summaries[0]
print(sample_summary)

Amanda baked cookies and will bring Jerry some tomorrow.


In [ ]:
sample_split = sample.split('\r\n')
print(sample_split)

['Amanda: I baked  cookies. Do you want some?', 'Jerry: Sure!', "Amanda: I'll bring you tomorrow :-)"]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
dialogueList = sample.split('\r\n')
summaryList = sample_summary.split('.')
print(dialogueList, summaryList)

['Amanda: I baked  cookies. Do you want some?', 'Jerry: Sure!', "Amanda: I'll bring you tomorrow :-)"] ['Amanda baked cookies and will bring Jerry some tomorrow', '']


In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
wholeList = dialogueList + summaryList
print(wholeList)

['Amanda: I baked  cookies. Do you want some?', 'Jerry: Sure!', "Amanda: I'll bring you tomorrow :-)", 'Amanda baked cookies and will bring Jerry some tomorrow', '']


In [ ]:
vectorizer.fit(wholeList)
dialogueVectors = vectorizer.transform(dialogueList)
summaryVectors = vectorizer.transform(summaryList)

In [ ]:
import numpy as np

In [ ]:
similarityMatrix = cosine_similarity(summaryVectors, dialogueVectors)

pairs = []
for i, row in enumerate(similarityMatrix):
    j = np.argmax(row)
    similarity = row[j]
    print(summaryList[i], " <-> ", dialogueList[j], ' : ', similarity)

Amanda baked cookies and will bring Jerry some tomorrow  <->  Amanda: I baked  cookies. Do you want some?  :  0.4238149396405716
  <->  Amanda: I baked  cookies. Do you want some?  :  0.0


In [ ]:
#sample_dataset, sample_summaries

In [ ]:
def seperate_summary(sample_dataset, sample_summaries):
    _dataset = {'text':[], 'summary':[]}
    _len = len(sample_dataset)
    count = 0
    for data, summary in zip(sample_dataset, sample_summaries):
        # print(data, summary)
        dialogueList = data.split('\r\n')
        if '.' in summary:
            summaryList = summary.split('.')[:-1]
        else:
            summaryList = [summary]
        wholeList = dialogueList + summaryList
        # print(wholeList)
        vectorizer = TfidfVectorizer()
        vectorizer.fit(wholeList)
        dialogueVectors = vectorizer.transform(dialogueList)
        summaryVectors = vectorizer.transform(summaryList)
        similarityMatrix = cosine_similarity(summaryVectors, dialogueVectors)
        boolList = [False for i in range(len(dialogueList))]
        for i, row in enumerate(similarityMatrix):
            j = np.argmax(row)
            boolList[j] = True
            _dataset['text'].append(dialogueList[j])
            _dataset['summary'].append(summaryList[i])
        for i in range(len(boolList)):
            if boolList[i] == True:
                continue
            _dataset['text'].append(dialogueList[i])
            _dataset['summary'].append('[]')
        count += 1
        if count % 100 == 0:
            print('Progress - ', count, '/', _len)
    return _dataset

In [ ]:
def view_dataset(dataset, length = 10):
    textDataset = dataset['text']
    summDataset = dataset['summary']
    i = 0
    for sentence, summary in zip(textDataset, summDataset):
        print('<Text> ', sentence, ' <Summary> ', summary)
        i += 1
        if i == length:
            break

In [ ]:
train_dataset = dataset['train']
processed_train_dataset = seperate_summary(train_dataset['dialogue'], train_dataset['summary'])

In [ ]:
len(train_dataset['dialogue'])

10

In [ ]:
view_dataset(processed_train_dataset, length = 20)

<Text>  Amanda: I baked  cookies. Do you want some?  <Summary>  Amanda baked cookies and will bring Jerry some tomorrow
<Text>  Jerry: Sure!  <Summary>  []
<Text>  Amanda: I'll bring you tomorrow :-)  <Summary>  []
<Text>  Olivia: Who are you voting for in this election?   <Summary>  Olivia and Olivier are voting for liberals in this election
<Text>  Oliver: Liberals as always.  <Summary>  []
<Text>  Olivia: Me too!!  <Summary>  []
<Text>  Oliver: Great  <Summary>  []
<Text>  Tim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores  <Summary>  Kim may try the pomodoro technique recommended by Tim to get more stuff done
<Text>  Tim: Hi, what's up?  <Summary>  []
<Text>  Kim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating  <Summary>  []
<Text>  Tim: What did you plan on doing?  <Summary>  []
<Text>  Kim: Oh you know, uni stuff and unfucking my room  <Summary>  []
<Text>  Kim: Maybe tomorrow I'll move my ass and do everything  <Su

In [ ]:
def save_dataset(dataset, file_path):
    with open(file_path, 'w') as file:
        textDataset = dataset['text']
        summDataset = dataset['summary']
        for sentence, summary in zip(textDataset, summDataset):
            file.write('<Text> '+ sentence + ' <Summary> ' + summary +'\n')

In [ ]:
final_dataset = {'train' : [], 'valid' : [], 'test' : []}

def add_dataset(dataset, mode):
    textDataset = dataset['text']
    summDataset = dataset['summary']
    for sentence, summary in zip(textDataset, summDataset):
        line = '<Text> '+ sentence + ' <Summary> ' + summary +'\n'
        final_dataset[mode].append(line)

In [ ]:
file_path = 'train.txt'
save_dataset(processed_train_dataset, file_path)

In [ ]:
valid_dataset = dataset['validation']
processed_valid_dataset = seperate_summary(valid_dataset['dialogue'], valid_dataset['summary'])

In [ ]:
test_dataset = dataset['test']
processed_test_dataset = seperate_summary(test_dataset['dialogue'], test_dataset['summary'])

In [ ]:
add_dataset(processed_train_dataset, 'train')
add_dataset(processed_valid_dataset, 'valid')
add_dataset(processed_test_dataset, 'test')

In [ ]:
!pip install parlai

In [ ]:
!parlai display_data -t msc:PersonaSummary --include-last-session True 

In [ ]:
!pip install names
!pip install gingerit

In [ ]:
import re
import json
# Read the JSON file
folder_path = '/usr/local/lib/python3.10/dist-packages/data/msc/msc/msc_personasummary'

# Data PreProcess Function
def createDataset(mode): 
    s1_data_path = f"{folder_path}/session_1/{mode}.txt"
    s2_data_path = f"{folder_path}/session_2/{mode}.txt"
    s3_data_path = f"{folder_path}/session_3/{mode}.txt"
    s4_data_path = f"{folder_path}/session_4/{mode}.txt"

    if mode == 'train':
        data_paths = [s1_data_path, s2_data_path, s3_data_path]
    else:
        data_paths = [s1_data_path, s2_data_path, s3_data_path, s4_data_path]

    mode_X = []
    mode_y = []
    # 데이터 파일 열기

    for data_path in data_paths:
        with open(data_path, 'r', encoding='utf-8') as f:
            for line in f:
                # 한 줄씩 읽기
                data = json.loads(line)
                id0_prev_count = 0
                id1_prev_count = 0
                # print(data['dialog'])
                for entry in data['dialog']:
                    bot_id = entry['id']
                    agg_persona_list = entry['agg_persona_list']
                    if bot_id == 'bot_0':
                        cur_agg_persona_list = agg_persona_list[id0_prev_count:]
                    else:
                        cur_agg_persona_list = agg_persona_list[id1_prev_count:]
                    text = entry['text']
                    summary = " ".join(cur_agg_persona_list)
                    if bot_id == 'bot_0':
                        id0_prev_count += len(cur_agg_persona_list)
                    else:
                        id1_prev_count += len(cur_agg_persona_list)

                    mode_X.append(text)
                    mode_y.append(summary)

    return mode_X, mode_y

In [ ]:
test_X, test_y = createDataset('test')

In [ ]:
msc_test_dataset = {'text' : test_X, 'summary' : test_y}

In [ ]:
train_X, train_y = createDataset('train')
msc_train_dataset = {'text' : train_X, 'summary' : train_y}

In [ ]:
valid_X, valid_y = createDataset('valid')
msc_valid_dataset = {'text': valid_X, 'summary' : valid_y}

In [ ]:
add_dataset(msc_train_dataset, 'train')
add_dataset(msc_valid_dataset, 'valid')
add_dataset(msc_test_dataset, 'test')

In [ ]:
def view_lines(lst, length=20):
    for i in range(length):
        print(lst[i])

In [ ]:
view_lines(final_dataset['train'])

In [ ]:
view_lines(final_dataset['train'])

In [ ]:
view_lines(final_dataset['valid'])

In [ ]:
view_lines(final_dataset['test'], length = len(final_dataset['test']))

In [ ]:
sen = "<Text> Gloria: This exam is a bit of a lottery in factGloria: You can't really get prepared, it's all about experienceEmma: But there are some rules and some typical texts right?Gloria: You can see some texts from previous yearsGloria: <file_other>Emma: Wow that's very usefulEmma: I have never seen this siteGloria: Yes it's very goodGloria: Actually it's good to read all the texts because you will see that some phrases repeat very oftenEmma: How much time do you have for all 4 parts?Gloria: 4 hoursEmma: Is it enough?Gloria: Well it has to beGloria: Would be perfect to have 2 more hours... But on the other hand it would be really exhaustingEmma: 4 hours and no breaks?Gloria: No breaks :/ So it's really important to be really focused and try to write as fast as you canGloria: And read it carefully and correct during the last hourEmma: I'm going to read everything from that website, it's great <Summary>  Emma sent her a link to a website with some texts from previous years so that she can prepare for the exam betterk"

In [ ]:
len(sen)

1029

In [ ]:
sen = "<Text> I was scrolling my email inbox this morning that I noticed an invitation for a music festival in my inbox. I will be held next weekend. It has written that there will be several music performances including country music. The invitation is for two individuals , so you can bring one guest with you. I'm not a fan of music and even if I was, I could not attend the event this weekend since I have to finalize a project. So I thought this might be interesting for you. <Summary> I do not like music. I have to work on a project this weekend. I received invitations to a country music festival for two people but I do not want to go. I want to give the tickets away."
len(sen)

670

In [ ]:
import random

for ds in final_dataset:
    random.shuffle(final_dataset[ds])

In [ ]:
def detect_long_sentences(lst):
    # 평균 길이보다 큰 길이를 갖는 인덱스 추출
    long_indices = [index for index, sentence in enumerate(lst) if len(sentence) > 800]
    return long_indices

In [ ]:
train_anormal_indices = detect_long_sentences(final_dataset['train'])
valid_anormal_indices = detect_long_sentences(final_dataset['valid'])
test_anormal_indices = detect_long_sentences(final_dataset['test'])

In [ ]:
len(final_dataset['test']), len(test_anormal_indices)

(32848, 35)

In [ ]:
def delete_indices(lst, index_list):
  for index in sorted(index_list, reverse=True):
      del lst[index]

In [ ]:
delete_indices(final_dataset['train'], train_anormal_indices)
delete_indices(final_dataset['valid'], valid_anormal_indices)
delete_indices(final_dataset['test'], test_anormal_indices)

In [ ]:
detect_long_sentences(final_dataset['test'])

[]

In [ ]:
import re

sentence = "<Text>  Amanda: I baked  cookies. Do you want some?  <Summary>  Amanda baked cookies and will bring Jerry some tomorrow"
result = re.sub(r'(?<=<Text> ) \w+:', '', sentence)
result

'<Text>  I baked  cookies. Do you want some?  <Summary>  Amanda baked cookies and will bring Jerry some tomorrow'

In [ ]:
sen = final_dataset['train'][0]
sen

'<Text> Amanda: I baked  cookies. Do you want some? <Summary> Amanda baked cookies and will bring Jerry some tomorrow\n'

In [ ]:
re.sub(r'(?<=<Text> )\w+:', '', sen)

'<Text>  I baked  cookies. Do you want some? <Summary> Amanda baked cookies and will bring Jerry some tomorrow\n'

In [ ]:
def erase_names(lst):
    processed_lst = []
    for i, sentence in enumerate(lst):
        processed_lst.append(re.sub(r'(?<=<Text> )\w+:', '', sentence))
    return processed_lst

In [ ]:
final_dataset['train'] = erase_names(final_dataset['train'])
final_dataset['valid'] = erase_names(final_dataset['valid'])

In [ ]:
final_dataset['test'] = erase_names(final_dataset['test'])

In [ ]:
view_lines(final_dataset['train'])

<Text>  I talked to my brother who had Nierzwicki 3 years ago - he failed half the class! <Summary>  3 years ago he failed half the class

<Text>  Hmmmm.. the new fella..Lucky you.. I've got an old one on The go lol.. I'll pop over with Luna for a cuppa.. Will you still be able to have her for a few days in Oct? <Summary>  Jenny Morris Sharpei will be able to have Luna for a few days in October

<Text> It also helps that it's cold here so not too much opportunity to splurge on amusements. <Summary> I work in finance. It is cold where I live.

<Text>  Maybe you could wait for me in that cafe nearby? My treat! <Summary> []

<Text> I've 2 pugs and a new kitten. They are still warming up to each other lol <Summary> I have 2 dogs and 1 cat.

<Text>  it's a strange story <Summary> []

<Text>  Will do! Love you! <Summary> []

<Text> Awesome, how many children do you have? <Summary> 

<Text> I think about it often and it makes me nervous, which seems crazy. How did your husband propose to you?

In [ ]:
len(final_dataset['train']), len(final_dataset['valid']), len(final_dataset['test'])

(281429, 33453, 32813)

In [ ]:
def save_dataset(lst, file_path):
    with open(file_path, "w") as file:
        for line in lst:
            file.write(line + "\n")

In [ ]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/Metabuddy/Data Process'

In [ ]:
file_path = f'{folder_path}/train.txt'
save_dataset(final_dataset['train'], file_path)
file_path = f'{folder_path}/valid.txt'
save_dataset(final_dataset['valid'], file_path)
file_path = f'{folder_path}/test.txt'
save_dataset(final_dataset['test'], file_path)